### Data generation

For the academic research writings of the specific authors are generated using these LLMs:
- gemini-pro
- mistralai/Mixtral-8x7B-Instruct-v0.1
- gpt-4

### Provider

Data is generated using [gpt4free](https://github.com/xtekky/gpt4free/tree/main), which implements reversed engineering technics in order to give free access to the most popular language models.

### Run

In order to run the notebook, few data is necessary to be filled

**GPT-4**

Create `hardir` in the root of the project. Save `.har` file explained in `.HAR File for OpenaiChat Provider` section in [gpt4free](https://github.com/xtekky/gpt4free/tree/main) README.

**Gemini-pro**

Create `hardir` in the root of the project and create `gemini_cookies.json` file:
```json
{
    "__Secure-1PSID": "your_cookies",
    "__Secure-1PSIDTS": "your_cookies"
}
```
How to find these cookies is explained in `Cookies` section in [gpt4free](https://github.com/xtekky/gpt4free/tree/main) README.

In [107]:
%pip install -U g4f[all]

Note: you may need to restart the kernel to use updated packages.


In [108]:
from g4f.client import Client
from g4f.cookies import set_cookies
from g4f.Provider import Gemini, OpenaiChat, DeepInfra
from datetime import datetime
import json
import os

In [109]:
res_dir = "res/"
har_dir = "hardir/"
gemini_cookies = f"{har_dir}gemini_cookies.json"
authors_list_filepath = f"{res_dir}gutenberg_author_list"
prompts_filepath = f"{res_dir}prompts"

models = [
    "gemini-pro", 
    "mistralai/Mixtral-8x7B-Instruct-v0.1", 
    "gpt-4"
]
provider =     {
    "gemini-pro": Gemini,
    "mistralai/Mixtral-8x7B-Instruct-v0.1": DeepInfra,
    "gpt-4": OpenaiChat
}
response_length = 5000

Setup necessary cookies

In [110]:
with open(gemini_cookies, "r") as f:
    cookies = json.load(f)
    set_cookies(".google.com", {
        "__Secure-1PSID": cookies["__Secure-1PSID"],
        "__Secure-1PSIDTS": cookies["__Secure-1PSIDTS"],
    })

In [111]:
def get_generated_response(system_spec, prompt, model):
    prompt = f"{prompt}. Use at least {response_length} words."
    client = Client()
    response = client.chat.completions.create(
        model=model,
        provider=provider[model],
        messages=[
            {"role": "system", "content": system_spec},
            {"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

In [112]:
def save_data(data):
    model_dir = f"{res_dir}{data['model'].replace('/', '_')}"
    data['id'] = int(datetime.now().timestamp())  
    filename = f"{model_dir}/{data['author']}/{data['id']}.json"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        f.write(json.dumps(data, indent=4))

Use the list of 10 authors with the highest total words count from `book_stats.ipynb`.

In [113]:
authors = [
    'George Alfred Henty',
    "Anthony Trollope",
    "Charlotte Mary Yonge",
    "Charles Dickens",
    "R M Ballantyne",
    "James Fenimore Cooper",
    "Robert Louis Stevenson",
    "William Dean Howells",
    "Henry Rider Haggard",
    "Sir Walter Scott"
]

In [114]:
prompts = open(prompts_filepath, 'r', encoding='utf-8').read().split('\n')
prompts[:5]

["Describe the most peculiar meal you've ever encountered.",
 'Explain the finer points of etiquette for a barnyard gathering.',
 'Convince a grumpy mule to pull a rickety carriage.',
 'Debate the merits of a mustache versus a beard.',
 'Write a love letter from a lovesick cactus to a blooming rose.']

In [115]:
for author in authors:
    system_spec = f"Come up with the answer in {author}'s writing style. Don't use direct references and citations of {author}. Answer in plain text format."
        
    for model in models:
        
        for prompt in prompts:
            print(f"Generating response for {author}'s response using {model} model.")
            response = get_generated_response(system_spec, prompt, model)

            data = {
                "requested_response_length": response_length,
                "response_length": len(response),
                "model": model,
                "created_at": datetime.now().isoformat(),
                "author": author, 
                "system_spec": system_spec,
                'prompt': prompt,
                'response':  response
            }

            save_data(data)

Generating response for George Alfred Henty's response using gemini-pro model.


RuntimeError: Invalid auth. SNlM0e not found